In [1]:
%autosave 100

Autosaving every 100 seconds


In [2]:
import numpy as np
import numpy.linalg as npl
from scipy import linalg as scl

In [3]:
import contextlib

@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    yield
    np.set_printoptions(**original)

In [4]:
MATRIX_SIZE = 10

In [5]:
def rand_matrix(N=MATRIX_SIZE, scale=100):
    return np.random.normal(scale=scale, size=(N,N))

def sym_matrix(N=MATRIX_SIZE, scale=100):
    m = rand_matrix(N, scale)
    return (m + m.T)/2

def diag_dominant_matrix(N=MATRIX_SIZE, scale=100):
    m = rand_matrix(N, scale)
    for i in range(N):
        m[i][i] = (abs(m[i]).sum()) * np.random.uniform(1, N)
    return m

def hilbert_matrix(N=MATRIX_SIZE, scale=100):
    return scl.hilbert(N)

def rand_vec(N=MATRIX_SIZE, scale=100):
    return np.random.normal(scale=scale, size=(N))

In [6]:
def cond_num(A):
    return npl.norm(A) * npl.norm(npl.inv(A))

In [7]:
def gauss(mat, vec):
    A = np.c_[mat, vec]
    N = vec.shape[0]
    
    for i in range(N):
        A[i] /= A[i][i]
        for j in range(i+1, N):
            A[j] -= A[i] * A[j][i]
        
    for i in range(N-1, -1, -1):
        for j in range(i-1, -1, -1):
            A[j] -= A[i] * A[j][i]

    return A[:,N]

In [8]:
def jacobi(mat, vec, eps=1e-8, max_iter=int(1e6)):
    a = mat.copy()
    b = vec.copy()
    N = b.shape[0]
    
    for i in range(N):
        b[i] /= a[i][i]
        a[i] /= -a[i][i]
        a[i][i] = 0
        
    x_ = b.copy()
    norm = 1
    iters = 0
    while norm > eps and iters < max_iter:
        x = (a @ x_) + b
        norm = npl.norm(x - x_)
        x_ = x
        iters += 1
    
    if (iters >= max_iter):
        print('Превышено максимальное число итераций. Текущая норма:', norm)
    else:
        print('Итераций:', iters)
    
    return x

In [9]:
def zeidel(mat, vec, eps=1e-8, max_iter=int(1e6), relax=True, omega=0.5):
    a = mat.copy()
    b = vec.copy()
    N = b.shape[0]
    
    for i in range(N):
        b[i] /= a[i][i]
        a[i] /= -a[i][i]
        a[i][i] = 0
        
    x_ = b.copy()
    x = np.zeros(b.shape)
    norm = 1
    iters = 0
    while norm > eps and iters < max_iter:
        x = x_.copy()
        for i in range(N):
            x[i] = (x[0:i] * a[i][0:i]).sum() + (x_[i:] * a[i][i:]).sum() + b[i]
        if relax:
            x = x*omega + x_*(1-omega)
        norm = npl.norm(x - x_)
        x_ = x
        iters += 1
    
    if (iters >= max_iter):
        print('Превышено максимальное число итераций. Текущая норма:', norm)
    else:
        print('Итераций:', iters)
    return x

In [10]:
def con_gradient(mat, vec, eps=1e-4, max_iter=int(1e6)):
    a = mat.copy()
    b = vec.copy()
    x_ = np.zeros(b.shape)
    iters = 0
    norm = 1
    
    while norm > eps and iters < max_iter:
        g = a @ x_ - b
        if iters == 0:
            r = -g
        else:
            r = -g + (((a @ r_) @ g) / ((a @ r_) @ r_))*r_
        s = - (g @ r)/((a @ r) @ r)
        x = x_ + s * r
        r_ = r
        x_ = x
        norm = npl.norm(r) / npl.norm(b)
        iters += 1
        
    if (iters >= max_iter):
        print('Превышено максимальное число итераций. Текущая норма:', norm)
    else:
        print('Итераций:', iters)
    return x

In [11]:
def solve_random(gen_mat, gen_vec, solver, N=MATRIX_SIZE, scale=100):
    solve(gen_mat(N, scale), gen_vec(N, scale), solver)
    
def solve(A, b, solver, print_log=True, **kwargs):
    with printoptions(precision=3, suppress=True): 
        if print_log:
            print('A')
            print(A)
            print('Число обусловленности A:', cond_num(A))
            print('b')
            print(b)
        x = solver(A,b, **kwargs)
        if print_log:
            print('x')
            print(x)
            print('Проверка Ax = b :', np.allclose(A @ x, b))
        return x

In [12]:
solve_random(rand_matrix, rand_vec, gauss)

A
[[ -64.287 -183.334   35.249   32.866   20.75   -98.766  -87.632  -68.504
   -10.141   53.875]
 [  -6.341   20.97   126.294   66.299  -93.836  -39.822   30.348 -198.05
   104.856    5.731]
 [-147.965 -160.199   29.499  205.732   34.862   87.859  -38.137  -61.103
  -125.45   -34.275]
 [  13.429  -18.093   23.327  -53.537    2.716   29.301 -175.154   48.642
    45.595  -83.067]
 [ -37.779   25.258   34.218 -193.529  -63.687  174.195   16.215  -69.497
    23.931   41.273]
 [ 203.68   127.345   11.897  -83.13    17.892   88.029   46.69    39.652
    71.852  -93.928]
 [  43.254 -109.338   30.984  -52.915   59.196   62.103   67.935  128.94
   117.871  147.917]
 [-194.871    7.977 -188.478  140.85  -212.967  -61.872  -81.062   63.074
  -251.012  -36.75 ]
 [   5.786  -96.753  133.225  -35.643  126.364  -83.691 -106.235  101.699
   108.247 -129.868]
 [ 131.642   26.536   44.806  -50.34    96.035    3.301   16.769 -122.381
  -259.012   56.997]]
Число обусловленности A: 27.2724981744
b
[-229.62

In [13]:
solve_random(diag_dominant_matrix, rand_vec, jacobi)

A
[[ 3450.014  -258.512  -134.74    -37.295   -73.099    29.255    27.361
     29.344    -7.286   -26.638]
 [ -181.6    2587.65   -123.222  -152.973    65.837   -37.101   -18.143
    166.956   -58.61    -49.863]
 [   79.032   -19.306  2912.056    94.289  -151.183   125.403  -129.995
     -2.674  -154.515   -74.615]
 [  198.875    59.488   -53.843  2692.221   -22.22    194.59     87.083
    -89.357    45.575    -7.609]
 [  -87.849   114.37    -63.377    90.194  5781.531   -70.802    68.359
     77.431   -27.345   -38.777]
 [    4.451   103.283    63.972   116.574   -81.831  4900.16    105.148
    -68.38     45.553    -8.058]
 [  176.405    -9.078   146.139   151.465   -74.319    75.631  6592.268
     28.63   -161.747    55.456]
 [ -129.008   -46.845  -104.752    96.972    66.462   -78.666   -79.492
   1425.779   252.622   126.575]
 [   76.252    97.408     0.936   -13.957    30.069   -77.748     9.709
     78.273   612.6     -66.971]
 [  147.427   -31.061    41.447   223.522  -134.796  

In [14]:
solve_random(diag_dominant_matrix, rand_vec, zeidel)

A
[[  5792.718   -150.3       26.384   -202.204     85.593    -71.795
     112.788    -29.181    -81.749    -25.978]
 [   -12.758   5252.099     99.633    -45.534    -47.973    135.       -71.041
      45.602    142.81      29.093]
 [   -58.786    197.891   2601.492     76.644    -49.298     33.195
      23.224     44.147    -59.441     88.519]
 [   108.918     97.578    126.089   6357.148     82.841   -132.824
    -164.154    -70.721     -3.769   -140.96 ]
 [   190.097    177.16    -146.528     33.053  11073.576    -20.694
      90.468   -238.79    -107.424     91.499]
 [   117.694    210.838     -5.33     106.476   -230.625   8002.31
     -89.686    -11.499     56.597     47.812]
 [    97.009    -95.052    -26.379    -86.683    148.492    -36.775
    7318.065   -183.429   -211.619    -91.364]
 [     8.588     -4.855   -250.789   -159.982     38.194     29.595
      37.715   1302.183     83.475     21.887]
 [  -114.89      30.63      39.365    133.438    -16.262   -136.429
    -105.92

In [15]:
solve_random(sym_matrix, rand_vec, con_gradient)

A
[[-141.575   67.055    6.766  -12.457  -34.319  -81.824  105.75  -172.102
    38.284  -56.029]
 [  67.055  -89.199   61.3     62.783  -60.329    7.244   -9.606  139.25
   -87.914   29.04 ]
 [   6.766   61.3    181.211  -28.052    8.33   -18.749  113.993  -34.085
    49.434  -15.877]
 [ -12.457   62.783  -28.052   97.531  -23.352  124.933   13.857    2.068
    70.374  -52.7  ]
 [ -34.319  -60.329    8.33   -23.352   21.831  -55.552   35.696   86.793
    19.154  -17.311]
 [ -81.824    7.244  -18.749  124.933  -55.552 -165.651  -60.378  -50.611
   -34.351   78.302]
 [ 105.75    -9.606  113.993   13.857   35.696  -60.378   17.536 -121.787
   -20.302  -12.28 ]
 [-172.102  139.25   -34.085    2.068   86.793  -50.611 -121.787    0.99
   -77.069 -103.724]
 [  38.284  -87.914   49.434   70.374   19.154  -34.351  -20.302  -77.069
  -113.6      6.591]
 [ -56.029   29.04   -15.877  -52.7    -17.311   78.302  -12.28  -103.724
     6.591   -0.242]]
Число обусловленности A: 35.4011869804
b
[ 173.03

In [16]:
solve_random(hilbert_matrix, rand_vec, con_gradient)

A
[[ 1.     0.5    0.333  0.25   0.2    0.167  0.143  0.125  0.111  0.1  ]
 [ 0.5    0.333  0.25   0.2    0.167  0.143  0.125  0.111  0.1    0.091]
 [ 0.333  0.25   0.2    0.167  0.143  0.125  0.111  0.1    0.091  0.083]
 [ 0.25   0.2    0.167  0.143  0.125  0.111  0.1    0.091  0.083  0.077]
 [ 0.2    0.167  0.143  0.125  0.111  0.1    0.091  0.083  0.077  0.071]
 [ 0.167  0.143  0.125  0.111  0.1    0.091  0.083  0.077  0.071  0.067]
 [ 0.143  0.125  0.111  0.1    0.091  0.083  0.077  0.071  0.067  0.062]
 [ 0.125  0.111  0.1    0.091  0.083  0.077  0.071  0.067  0.062  0.059]
 [ 0.111  0.1    0.091  0.083  0.077  0.071  0.067  0.062  0.059  0.056]
 [ 0.1    0.091  0.083  0.077  0.071  0.067  0.062  0.059  0.056  0.053]]
Число обусловленности A: 1.63321977091e+13
b
[-152.105   69.356  -57.915  249.15    -2.568 -103.522  130.613 -173.132
   60.712  -30.766]
Превышено максимальное число итераций. Текущая норма: 0.815786251079
x
[  3.551e+09  -3.105e+11   6.671e+12  -6.105e+13   2.927e+

In [17]:
A = hilbert_matrix(N=10, scale=10)
b = rand_vec(10, 10)
print(solve(A, b, gauss, False))
print(solve(A, b, con_gradient, print_log=False))

[ -1.27622656e+08   1.09412458e+10  -2.31747644e+11   2.09796555e+12
  -9.97384597e+12   2.73447515e+13  -4.47650404e+13   4.31794199e+13
  -2.26325802e+13   4.97034868e+12]
Превышено максимальное число итераций. Текущая норма: 1.59773036352
[ -1.27442287e+08   1.09255517e+10  -2.31411602e+11   2.09489810e+12
  -9.95916708e+12   2.73042917e+13  -4.46985109e+13   4.31150061e+13
  -2.25987091e+13   4.96288949e+12]
